In [2]:
import os
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

dataset_path = r"C:\Users\sagni\Downloads\archive (1)\archive\animals\animals"
output_csv = "animal_tracking_data.csv"

poaching_keywords = ["human", "gun", "human_with_gun"]

base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")

data = []
labels = []

for label_folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, label_folder)
    if not os.path.isdir(folder_path):
        continue
    
    poaching_detected = 1 if any(pk in label_folder.lower() for pk in poaching_keywords) else 0
    
    for file_name in os.listdir(folder_path):
        if not (file_name.lower().endswith(".jpg") or file_name.lower().endswith(".png")):
            continue
        
        file_path = os.path.join(folder_path, file_name)
        image = cv2.imread(file_path)
        if image is None:
            continue
        
        image = cv2.resize(image, (224, 224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        
        features = base_model.predict(image, verbose=0)[0]
        
        data.append(features)
        labels.append(poaching_detected)

feature_names = [f"f{i}" for i in range(len(data[0]))]
df = pd.DataFrame(data, columns=feature_names)
df["poaching_detected"] = labels

df.to_csv(output_csv, index=False)
print(f"Dataset saved to {output_csv} with {len(df)} samples.")


C:\Users\sagni\AppData\Local\Temp\ipykernel_14248\3701344960.py:17: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


Dataset saved to animal_tracking_data.csv with 2543 samples.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib
import cv2
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

# ===== STEP 1: Load dataset =====
df = pd.read_csv("animal_tracking_data.csv")

X = df.drop(columns=["poaching_detected"]).values
y = df["poaching_detected"].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ===== STEP 2: Train model =====
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["No Poaching", "Poaching"]))

# Save model
joblib.dump(clf, "poaching_detector.pkl")
print("Model saved to poaching_detector.pkl")

# ===== STEP 3: Predict function =====
def predict_poaching(image_path):
    base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")
    
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    image = cv2.resize(image, (224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    
    features = base_model.predict(image, verbose=0)[0].reshape(1, -1)
    
    model = joblib.load("poaching_detector.pkl")
    pred = model.predict(features)[0]
    confidence = model.predict_proba(features)[0][pred]
    
    if pred == 1:
        return f"🚨 Poaching Detected! (Confidence: {confidence:.2f})"
    else:
        return f"✅ No Poaching (Confidence: {confidence:.2f})"

# ===== STEP 4: Test prediction on Human_with_gun =====
human_with_gun_dir = r"C:\Users\sagni\Downloads\archive (1)\archive\animals\animals\Human_with_gun"
test_image = os.path.join(human_with_gun_dir, os.listdir(human_with_gun_dir)[0])  # first image in folder

print(f"Testing on: {test_image}")
print(predict_poaching(test_image))


              precision    recall  f1-score   support

 No Poaching       0.96      1.00      0.98       455
    Poaching       1.00      0.63      0.77        54

    accuracy                           0.96       509
   macro avg       0.98      0.81      0.88       509
weighted avg       0.96      0.96      0.96       509

Model saved to poaching_detector.pkl
Testing on: C:\Users\sagni\Downloads\archive (1)\archive\animals\animals\Human_with_gun\austin-pacheco-gEsSvM1Vm8g-unsplash.jpg


C:\Users\sagni\AppData\Local\Temp\ipykernel_14248\3382365929.py:36: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights="imagenet", include_top=False, pooling="avg")


🚨 Poaching Detected! (Confidence: 0.78)
